In [2]:
import pandas as pd

# Data preprocessing

In [4]:
# Import imdb data
data_folder = './raw_data/'
imdb_rating_path = data_folder + 'imdb/'
cmu_path = data_folder + 'cmu_movies/'
baby_names_path = data_folder + 'usa_baby_names/'

rating_df = pd.read_csv(imdb_rating_path + 'imdb_rating.tsv', sep='\t', index_col='tconst')
display(rating_df)

,averageRating,numVotes
tconst,,
tt0000001,5.7,2004
tt0000002,5.8,269
tt0000003,6.5,1904
tt0000004,5.5,178
tt0000005,6.2,2685
...,...,...
tt9916730,7.6,11
tt9916766,7.0,22
tt9916778,7.2,36
